In [ ]:
!git clone https://github.com/23lnlx/itmo-kaggle-inclass.git

Cloning into 'itmo-kaggle-inclass'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 10), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c itmo-acoustic-event-detection-2022

100% 7.31G/7.34G [01:06<00:00, 110MB/s]
100% 7.34G/7.34G [01:06<00:00, 119MB/s]


In [ ]:
! mkdir kdata


In [ ]:
! unzip itmo-acoustic-event-detection-2022.zip -d kdata

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: kdata/audio_train/train/1e1d3e022f56c156b4c7.wav  
  inflating: kdata/audio_train/train/1e280f74b308b6f20507.wav  
  inflating: kdata/audio_train/train/1e3bb494145d29683d5b.wav  
  inflating: kdata/audio_train/train/1e4626b1264b16cb8bfe.wav  
  inflating: kdata/audio_train/train/1e5494668fe5055092a0.wav  
  inflating: kdata/audio_train/train/1e57c2be7296212fbe9e.wav  
  inflating: kdata/audio_train/train/1e616116120b9706516a.wav  
  inflating: kdata/audio_train/train/1e6fbb2e48198873d22e.wav  
  inflating: kdata/audio_train/train/1e7a81713bb0e482a21b.wav  
  inflating: kdata/audio_train/train/1e7d1b5f4c78d78353e2.wav  
  inflating: kdata/audio_train/train/1e7f2f384ee98e31345d.wav  
  inflating: kdata/audio_train/train/1e82f46834ec5949995b.wav  
  inflating: kdata/audio_train/train/1e917fa34ac2fecb5eed.wav  
  inflating: kdata/audio_train/train/1ea0ac9647c2e889e62d.wav  
  inflating: kdata/audio_train/train

In [ ]:
! pip install efficientnet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=8665a66450667a2eb167d8a9352402eddca8ee75cb22d0fb713220804a43fff4
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
!pip install pytorchtools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# path to your train/test/meta folders
DATA_PATH = '/content/kdata/'

# names of valuable files/folders
train_meta_fname = 'train.csv'
test_meta_fname = 'sample_submission.csv'
train_data_folder = 'audio_train/train'
test_data_folder = 'audio_test/test'

In [ ]:
! pip install torchaudio torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
import torchvision
from torchaudio import transforms
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import CyclicLR
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

In [ ]:
# set seeds
import random

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
df_train = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_test = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [ ]:
n_classes = df_train.label.nunique()
print(n_classes)
classes_dict = {cl:i for i,cl in enumerate(df_train.label.unique())}
df_train['label_encoded'] = df_train.label.map(classes_dict)
df_train.head()

41


,fname,label,label_encoded
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping,0
1,00d77b917e241afa06f1.wav,Squeak,1
2,17bb93b73b8e79234cb3.wav,Electric_piano,2
3,7d5c7a40a936136da55e.wav,Harmonica,3
4,17e0ee7565a33d6c2326.wav,Snare_drum,4


In [ ]:
# https://github.com/lukemelas/EfficientNet-PyTorch
class BaseLineModel(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        # self.ms = torchaudio.transforms.MelSpectrogram(sample_rate)
#         self.bn1 = nn.BatchNorm2d(1)
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=3, kernel_size=3, padding=1)
        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')
        # use it as features
#         for param in self.features.parameters():
#             param.requires_grad = False
            
        self.lin1 = nn.Linear(1000, 333)
        
        self.lin2 = nn.Linear(333, 111)
                
        self.lin3 = nn.Linear(111, n_classes)
        
    def forward(self, x):
        # x = self.ms(x)
#         x = self.bn1(x)
                
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))
        
        x = self.features(x)

        x = x.view(x.shape[0], -1)
        x = F.relu(x)
        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [ ]:
def sample_or_pad(waveform, wav_len=32000):
    m, n = waveform.shape
    if n < wav_len:
        padded_wav = torch.zeros(1, wav_len)
        padded_wav[:, :n] = waveform
        return padded_wav
    elif n > wav_len:
        offset = np.random.randint(0, n - wav_len)
        sampled_wav = waveform[:, offset:offset+wav_len]
        return sampled_wav
    else:
        return waveform

# class EventDetectionDataset(Dataset):
#     def __init__(self, data_path, x, y=None):
#         self.x = x
#         self.y = y
#         self.data_path = data_path
#         self.mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=16000)
    
#     def __len__(self):
#         return len(self.x)

#     def __getitem__(self, idx):
#         path2wav = os.path.join(self.data_path, self.x[idx])
#         waveform, sample_rate = torchaudio.load(path2wav, normalize=True)
#         waveform = sample_or_pad(waveform)
#         spec = self.mel_spectrogram(waveform)
#         label = torch.zeros(41)
#         label[self.y[idx]] = 1.
#         if self.y is not None:
#             # return waveform, self.y[idx]
#             if self.train and idx > 0 and idx%5 == 0:

# #             # Choose another image/label randomly
#             mixup_idx = random.randint(0, len(self.x)-1)
#             path2wav = os.path.join(self.data_path, self.x[mixup_idx])
#             mixup_waveform, sample_rate = torchaudio.load(path2wav, normalize=True)
#             mixup_waveform = sample_or_pad(mixup_waveform)
#             return spec, label
#         return waveform
        
class EventDetectionDataset(Dataset):
    def __init__(self, data_path, x, y=None, sr=16000, train=False):
        self.x = x
        self.y = y
        self.data_path = data_path
        self.train = train
        self.mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=sr)
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        path2wav = os.path.join(self.data_path, self.x[idx])
        waveform, sample_rate = torchaudio.load(path2wav, normalize=True)
        waveform = sample_or_pad(waveform)
        
        spec = self.mel_spectrogram(waveform)
        # print(type(spec), type(label))
        if self.y is not None:
          label = torch.zeros(41)
          label[self.y[idx]] = 1.
          # print(idx)
          if self.train and idx > 0 and idx%5 == 0:

            # Choose another image/label randomly
            mixup_idx = random.randint(0, len(self.x)-1)
            path2wav = os.path.join(self.data_path, self.x[mixup_idx])
            mixup_waveform, sample_rate = torchaudio.load(path2wav, normalize=True)
            mixup_waveform = sample_or_pad(mixup_waveform)

            mixup_label = torch.zeros(41)
            mixup_label[self.y[mixup_idx]] = 1.
            mixup_spec = self.mel_spectrogram(mixup_waveform)
            # if self.transform:
                # mixup_image = transform(self.x[mixup_idx])

            # Select a random number from the given beta distribution
            # Mixup the images accordingly
            alpha = 0.2
            lam = np.random.beta(alpha, alpha)
            spec = lam * spec + (1 - lam) * mixup_spec
            label = lam * label + (1 - lam) * mixup_label
            # print(type(spec), type(label))
          return spec, label
        return spec

In [ ]:
sr = 16000
X_train, X_val, y_train, y_val = train_test_split(df_train.fname.values, df_train.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train, train=True),
                        batch_size=41
                )
val_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )
test_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, test_data_folder), df_test.fname.values, None),
                        batch_size=41, shuffle=False
                )

In [ ]:
def eval_model(model, eval_dataset):
    model.eval()
    forecast, true_labs = [], []
    with torch.no_grad():
        for wavs, labs in tqdm(eval_dataset):
            wavs, labs = wavs.cuda(), labs.detach().numpy().argmax(axis=1)
            true_labs.append(labs)
            outputs = model.inference(wavs)
            
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labs = [x for sublist in true_labs for x in sublist]
    return f1_score(forecast, true_labs, average='macro')

In [ ]:
criterion = nn.CrossEntropyLoss()
model = BaseLineModel()
model = model.cuda()
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Loaded pretrained weights for efficientnet-b0


In [ ]:
n_epoch = 100
best_f1 = 0
for epoch in range(n_epoch):
    model.train()
    for wavs, labs in tqdm(train_loader):
        optimizer.zero_grad()
        wavs, labs = wavs.cuda(), labs.cuda()
        outputs = model(wavs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
#     if epoch % 10 == 0:
    f1 = eval_model(model, val_loader)
    f1_train = eval_model(model, train_loader)
    print(f'epoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), '../baseline_fulldiv.pt')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

100%|██████████| 111/111 [00:16<00:00,  6.71it/s]


epoch: 0, f1_test: 0.10881514510486336, f1_train: 0.12491591320488005


100%|██████████| 111/111 [00:16<00:00,  6.61it/s]


epoch: 1, f1_test: 0.10223543943360824, f1_train: 0.10405710509574878


100%|██████████| 111/111 [00:17<00:00,  6.51it/s]


...

epoch: 97, f1_test: 0.6204840957233853, f1_train: 0.8453644466446095


100%|██████████| 111/111 [00:16<00:00,  6.74it/s]


epoch: 98, f1_test: 0.6064934371167476, f1_train: 0.8401521920312207


100%|██████████| 111/111 [00:16<00:00,  6.57it/s]

epoch: 99, f1_test: 0.6206432645738215, f1_train: 0.8515107546033077


In [ ]:
# baseline
n_epoch = 100
best_f1 = 0
for epoch in range(n_epoch):
    model.train()
    for wavs, labs in tqdm(train_loader):
        optimizer.zero_grad()
        wavs, labs = wavs.cuda(), labs.cuda()
        outputs = model(wavs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
#     if epoch % 10 == 0:
    f1 = eval_model(model, val_loader)
    f1_train = eval_model(model, train_loader)
    print(f'epoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), '../baseline_fulldiv.pt')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

100%|██████████| 111/111 [00:31<00:00,  3.48it/s]


epoch: 0, f1_test: 0.0684413973377112, f1_train: 0.08036563227922518


100%|██████████| 111/111 [00:15<00:00,  7.08it/s]


epoch: 1, f1_test: 0.2409412249139456, f1_train: 0.2667595934895602


100%|██████████| 111/111 [00:11<00:00,  9.84it/s]


...

epoch: 97, f1_test: 0.5917657834821447, f1_train: 0.8107191299161887


100%|██████████| 111/111 [00:10<00:00, 10.30it/s]


epoch: 98, f1_test: 0.6235056103183135, f1_train: 0.8081159915835898


100%|██████████| 111/111 [00:11<00:00,  9.83it/s]

epoch: 99, f1_test: 0.5931228546664078, f1_train: 0.8091103515043455


In [ ]:
# change of baseline model
class ModelV2(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2)
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.cnn2extra = nn.Conv2d(in_channels=64, out_channels=20, kernel_size=1, padding=0)
        
        self.cnn3 = nn.Conv2d(in_channels=20, out_channels=3, kernel_size=3, padding=1)
        
        self.max_pool = nn.MaxPool2d(3, 2)
       
        self.features = EfficientNet.from_pretrained('efficientnet-b0')
            
        self.lin1 = nn.Linear(1000, 333)
        
        self.lin2 = nn.Linear(333, 111)
                
        self.lin3 = nn.Linear(111, n_classes)

        self.drop_layer = nn.Dropout(p=0.4) 

        self.cnn1_bn = nn.BatchNorm2d(32)
        self.cnn2_bn = nn.BatchNorm2d(64)
        self.cnn3_bn = nn.BatchNorm2d(3)
        self.lin1_bn = nn.BatchNorm1d(333)
        self.lin2_bn = nn.BatchNorm1d(111)
        
    def forward(self, x):
        x = self.ms(x)
                
        x = F.relu(self.cnn1_bn(self.cnn1(x)))
        x = F.relu(self.cnn2_bn(self.cnn2(x)))
        x = self.max_pool(x)
        x = F.relu(self.cnn2extra(x))
        x = F.relu(self.cnn3_bn(self.cnn3(x)))

        x = self.features(x)

        x = x.view(x.shape[0], -1)
        x = self.drop_layer(x)
        x = F.relu(self.lin1_bn(self.lin1(x)))
        x = F.relu(self.lin2_bn(self.lin2(x)))
        x = self.lin3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [ ]:
criterion2 = nn.CrossEntropyLoss()
model2 = ModelV2()
model2 = model2.cuda()
lr = 1e-3

optimizer2 = torch.optim.Adam(model2.parameters(), lr=lr)
# scheduler = CyclicLR(optimizer2, base_lr=lr/2, max_lr=lr, step_size_up=80, step_size_down=(len(train_loader) - 80))

In [3]:
n_epoch = 100
best_f1 = 0
for epoch in range(n_epoch):
    model2.train()
    for wavs, labs in tqdm(train_loader):
        optimizer2.zero_grad()
        wavs, labs = wavs.cuda(), labs.cuda()
        outputs = model2(wavs)
        loss = criterion2(outputs, labs)
        loss.backward()
        optimizer2.step()

        # scheduler.step()
#     if epoch % 10 == 0:
    f1 = eval_model(model2, val_loader)
    f1_train = eval_model(model2, train_loader)
    print(f'epoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model2.state_dict(), '../baseline_fulldiv.pt')
        
    lr = lr * 0.97
    for param_group in optimizer2.param_groups:
        param_group['lr'] = lr

100%|██████████| 111/111 [00:14<00:00,  7.66it/s]
epoch: 0, f1_test: 0.07697615112313103, f1_train: 0.07826489608877954
100%|██████████| 111/111 [00:14<00:00,  7.65it/s]
epoch: 1, f1_test: 0.11683850869904917, f1_train: 0.11628960729452452
100%|██████████| 111/111 [00:14<00:00,  7.70it/s]
epoch: 2, f1_test: 0.2154387726609989, f1_train: 0.2274105794427145
100%|██████████| 111/111 [00:14<00:00,  7.65it/s]
epoch: 3, f1_test: 0.20669931918895773, f1_train: 0.24261859882051948
100%|██████████| 111/111 [00:14<00:00,  7.65it/s]
epoch: 4, f1_test: 0.22127056803982595, f1_train: 0.2350701833177046
100%|██████████| 111/111 [00:14<00:00,  7.70it/s]
epoch: 5, f1_test: 0.4143568490412995, f1_train: 0.4447747595370425
100%|██████████| 111/111 [00:14<00:00,  7.67it/s]
epoch: 6, f1_test: 0.279094088237962, f1_train: 0.3443336355847003
100%|██████████| 111/111 [00:14<00:00,  7.57it/s]
epoch: 7, f1_test: 0.3047462403599801, f1_train: 0.36571036476539914
100%|██████████| 111/111 [00:14<00:00,  7.63it/s]

In [ ]:
# make a model
model_name = 'model2_fulldiv.pt'
model = ModelV2().cuda()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast = []
with torch.no_grad():
    for wavs in tqdm(test_loader):
        wavs = wavs.cuda()
        outputs = model.inference(wavs)
        outputs = outputs.detach().cpu().numpy().argmax(axis=1)
        forecast.append(outputs)
forecast = [x for sublist in forecast for x in sublist]
decoder = {classes_dict[cl]:cl for cl in classes_dict}
forecast = pd.Series(forecast).map(decoder)
df_test['label'] = forecast
df_test.to_csv(f'{model_name}.csv', index=None)

Loaded pretrained weights for efficientnet-b0


100%|██████████| 93/93 [00:29<00:00,  3.18it/s]
